In [1]:
function generateSudoku(N)
    i=0
    sudoku = zeros(Int64, 9, 9)
    map = zeros(Int64, 9, 9)
    while i<N
        x=rand(1:9)
        y=rand(1:9)
        val = rand(1:9)
        
        if sudoku[x,y]==0
            sudoku[x,y]=val
            map[x,y]=1
            i+=1
        end
    end
    return sudoku, map
end

generateSudoku (generic function with 1 method)

In [2]:
function randomFilling(sudoku)
    counters= zeros(Int64, 9)
    for i=1:9
        for j=1:9
            if sudoku[i, j]!=0
                counters[sudoku[i,j]]+=1
            end
        end
    end
    
    marker=1
    for i=1:9
        for j=1:9
            if sudoku[i, j]==0
                while counters[marker]==9
                    marker+=1
                end
                sudoku[i,j]=marker
                counters[marker]+=1
            end
        end
    end
    return sudoku
end

randomFilling (generic function with 1 method)

In [3]:
function calculateEnergy(sudoku)
    sum=243
    #rows
    for i=1:9
        flagArray=zeros(Int64, 9)
        for j=1:9
            value = sudoku[i, j]
            flagArray[value]=1
        end
        for j=1:9
            sum-=flagArray[j]
        end
    end
    
    #columns
    for i=1:9
        flagArray=zeros(Int64, 9)
        for j=1:9
            value=sudoku[j, i]
            flagArray[value]=1
        end
        for j=1:9
            sum-=flagArray[j]
        end
    end
    
    
    HeightStep=[1,4,7]
    WidthStep=[1,4,7]
    #squares
    heightCount = 1
    for i=1:3
        widthCount = 1
        for j=1:3
            flagArray=zeros(Int64, 9)
            for k=HeightStep[heightCount]:HeightStep[heightCount]+2
                for l=WidthStep[widthCount]:WidthStep[widthCount]+2
                    value=sudoku[k, l]
                    flagArray[value]=1
                end
            end
            for k=1:9
                sum-=flagArray[k]
            end
            widthCount +=1
        end
        heightCount +=1
    end
    return 3^sum
end

calculateEnergy (generic function with 1 method)

In [37]:
function simulatedAnnealing(sudoku, map, T)
    while T>=1
        x0=rand(1:9)
        y0=rand(1:9)
        while(map[x0, y0]==1)
            x0=rand(1:9)
            y0=rand(1:9)
        end
        signArray=[-1 -1; -1 0; -1 1; 0 -1; 0 0; 0 1; 1 -1 ; 1 0 ;1 1]
        bestSign=0
        bestSudoku = copy(sudoku)
        bestEnergy = typemax(Float64)
        
        for i=1:9
            if x0+signArray[i,1] >= 1 && x0 + signArray[i,1] <=9
                if y0 + signArray[i,2] >= 1 && y0 + signArray[i,2] <=9
                    if map[x0+signArray[i,1], y0+signArray[i,2]] != 1
                        tmpSudoku = copy(sudoku)
                        tmpSudoku[x0+signArray[i,1], y0+signArray[i,2]] = sudoku[x0, y0]
                        tmpSudoku[x0, y0] = sudoku[x0+signArray[i,1], y0+signArray[i,2]]
                        calculatedEnergy = calculateEnergy(sudoku)
                        if calculatedEnergy < bestEnergy
                            bestEnergy = calculatedEnergy
                            bestSudoku = copy(tmpSudoku)
                            bestSign = i
                        end
                    end
                end
            end
        end
        
        delta = calculateEnergy(bestSudoku) - calculateEnergy(sudoku)
        probability = exp(-delta/T)
        u=rand(Float64)
        if delta < 0 
            sudoku = copy(bestSudoku)
        elseif probability > u
            sudoku = copy(bestSudoku)
        end
        T*=0.9999
    end
    return sudoku
end

simulatedAnnealing (generic function with 1 method)

In [38]:
function simulatedAnnealing2(sudoku, map, T)
    while T>=1
        x0=rand(1:9)
        y0=rand(1:9)
        while(map[x0, y0]==1)
            x0=rand(1:9)
            y0=rand(1:9)
        end
        bestSudoku = copy(sudoku)
        bestEnergy = typemax(Float64)
        
        for i=1:9
            for j=1:9
                if i!=x0 && j!=y0
                    if map[i, j]!=1
                        tmpSudoku = copy(sudoku)
                        tmpSudoku[i, j] = sudoku[x0, y0]
                        tmpSudoku[x0, y0] = sudoku[i, j]
                        calculatedEnergy = calculateEnergy(sudoku)
                        if calculatedEnergy < bestEnergy
                            bestEnergy = calculatedEnergy
                            bestSudoku = copy(tmpSudoku)
                        end
                    end
                end
            end
        end
        
        delta = calculateEnergy(bestSudoku) - calculateEnergy(sudoku)
        probability = exp(-delta/T)
        u=rand(Float64)
        if delta < 0 
            sudoku = copy(bestSudoku)
        elseif probability > u
            sudoku = copy(bestSudoku)
        end
        T*=0.9999
    end
    return sudoku
end

simulatedAnnealing2 (generic function with 1 method)

In [39]:
N=7
sudoku, map = generateSudoku(N)
sudoku

9×9 Array{Int64,2}:
 0  0  0  0  0  0  0  7  0
 0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  8  0
 0  0  0  0  0  6  0  0  0
 0  0  0  0  3  0  0  3  0
 0  0  3  7  0  0  0  0  0
 0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0

In [40]:
sudoku = randomFilling(sudoku)
println(calculateEnergy(sudoku))
sudoku

117


9×9 Array{Int64,2}:
 1  1  1  1  1  1  1  7  1
 1  2  2  2  2  2  2  2  2
 2  3  3  3  3  3  3  8  4
 4  4  4  4  4  6  4  4  4
 5  5  5  5  3  5  5  3  5
 5  5  3  7  6  6  6  6  6
 6  6  6  7  7  7  7  7  7
 7  8  8  8  8  8  8  8  8
 9  9  9  9  9  9  9  9  9

In [60]:
T=100
resSudoku = simulatedAnnealing(sudoku, map, T)
println(calculateEnergy(resSudoku))
resSudoku

56


9×9 Array{Int64,2}:
 6  6  4  2  5  9  6  7  2
 9  1  5  1  7  8  2  3  1
 4  3  9  5  4  7  8  8  1
 7  3  9  2  1  6  2  4  8
 8  6  1  8  3  5  4  3  7
 4  5  3  7  2  3  9  6  2
 2  1  8  4  9  2  9  7  6
 5  5  6  9  3  4  1  6  7
 3  9  4  5  8  1  7  5  8

In [63]:
T=100
resSudoku2 = simulatedAnnealing2(sudoku, map, T)
println(calculateEnergy(resSudoku2))
resSudoku2

55


9×9 Array{Int64,2}:
 7  1  2  5  1  1  1  7  4
 1  3  9  6  4  9  3  5  6
 6  5  4  3  2  8  9  8  3
 1  8  4  2  7  6  7  6  9
 7  8  9  4  3  7  2  3  5
 9  6  3  7  9  2  6  8  4
 1  2  1  9  5  5  6  3  2
 5  7  8  5  8  3  4  2  8
 1  8  6  2  4  4  5  7  9